In [20]:
import numpy as np
import pandas as pd
from numpy import dot
from numpy.linalg import norm
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
import string
import gensim
from gensim.models import Word2Vec
from gensim.models import FastText

In [109]:
def cos_sim(a,b):
    return round(dot(a,b)/(norm(a)*norm(b)),3)

In [6]:
# Text Preprocessing
# function to remove all urls 
def remove_urls(text):
    new_text = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^A-Z0-9a-z \t])|(\w+:\/\/\S+)"," ",text).split())
    return new_text

# make text lower case
def lower_case(text):
    text = text.lower()
    return text

# remove numbers
def remove_numbers(text):
    result = re.sub(r'\d+','',text)
    return result

# remove punctuation
def remove_punctuation(text):
    translator = str.maketrans('','',string.punctuation)
    text = text.translate(translator)
    return text

# tokenize
def tokenize(text):
    text = word_tokenize(text)
    return text

# remove stopwords
def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    text = [i for i in text if i not in stop_words]
    return text

# lemmatize text
lemmatizer = WordNetLemmatizer()
def lemmatize(text):
    text = [lemmatizer.lemmatize(token) for token in text]
    return text

# preprocessing the text
def preprocessing(text):

    text = lower_case(text)
    text = remove_urls(text)
    text = remove_numbers(text)
    text = remove_punctuation(text)
    text = tokenize(text)
    text = remove_stopwords(text)
    text = lemmatize(text)
    text = ' '.join(text)
    
    return text

In [18]:
vector_size = 100

In [21]:
skipgram_100 = Word2Vec.load("skipgram_100.bin")
fast_n_100 = Word2Vec.load("fast_n_100.bin")

In [34]:
def get_mean_vector(model,words):
    words = [word for word in word_tokenize(words) if word in list(model.wv.index_to_key)] #Add the word to list of words if found in vocab
    if len(words)>=1:
        return np.mean(model.wv[words], axis=0)
    else:
        return np.zeros([vector_size])

In [129]:
# input query processing
def preprocessing_input(query,model):
    query = preprocessing(query)
    query=query.replace('\n',' ')
    k = get_mean_vector(model,query)
    return k



In [37]:
pd.set_option("display.max_colwidth", -1)

C:\Users\Gyegn\AppData\Local\Temp\ipykernel_30972\2475433900.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option("display.max_colwidth", -1)


In [38]:
with open("dimensions-covid19-export-2021-09-01-h15-01-02_clinical_trials.csv","r",encoding="utf-8",errors="ignore") as f:
    content = f.read()

In [39]:
from io import StringIO
df = pd.read_csv(StringIO(content))

C:\Users\Gyegn\AppData\Local\Temp\ipykernel_30972\2187133585.py:2: DtypeWarning: Columns (21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,133,134,135,136,137,138,139,140,143,144) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(StringIO(content))


In [44]:
df1 = df[["Date added","Trial ID","Title","Brief title","Acronym","Abstract"]]

In [45]:
df1.head()

Date added     Trial ID  \
0  9/1/2021   NCT05029245   
1  9/1/2021   NCT05029037   
2  9/1/2021   NCT05029011   
3  9/1/2021   NCT05028998   
4  9/1/2021   NCT05028881   

                                                                                                                                                                                                         Title  \
0  The 8-week, Prospective, Randomized Controlled of IntraDermal Administration of Comirnaty® 6 Microgram Versus Intramuscular Comirnaty® 30 Microgram by 28 Days Interval Efficacy Study in Healthy Volunteer   
1  High-dose Intravenous Vitamin C (HDIVC) as Adjuvant Therapy in Critical Patients With Positive COVID-19. A Pilot Randomized Controlled Dose-comparison Trial.                                                 
2  Low-cost Sensor System for COVID-19 Patient Monitoring: Validation of MediByte VTS                                                                                                                            
3  The Impact of COVID-19-related Medication Assisted Treatment Policy Changes on Care and Outcomes for Patients With Opioid Use Disorder                                                                        
4  Serological Responses to SARS-CoV-2 and Their Temporal Pattern in HIV Infected Persons                                                                                                                        

                                                                                                                                                     Brief title  \
0  IntraDermal Versus Intramuscular Comirnaty® Efficacy Study                                                                                                      
1  High-dose Intravenous Vitamin C (HDIVC) as Adjuvant Therapy in Critical Patients With Positive COVID-19. A Pilot Randomized Controlled Dose-comparison Trial.   
2  Low-cost Sensor System for COVID-19 Patient Monitoring: Validation of MediByte VTS                                                                              
3  Covid-related Opioid Treatment Policy Evaluation                                                                                                                
4  Hong Kong HIV SARS-CoV-2 Serology                                                                                                                               

  Acronym  \
0  PRIDE    
1  HDIVC    
2  NaN      
3  COPE     
4  NaN      

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

In [50]:
df1.shape

(12126, 6)

In [48]:
# Load the average vectors of the abstract
vector_skipgram_100 = pd.read_csv("method2_k_skipgram_100.csv")

In [76]:
# preprocessing the average vectors of the abstract
vector_skipgram_100 = vector_skipgram_100.T

In [80]:
skipgram_100_vectors = []
for i in range(df1.shape[0]):
    skipgram_100_vectors.append(vector_skipgram_100[i].values)

In [130]:
# Function to retrieve top n similar results
def top_n(query,p,df1,model):

    query = preprocessing_input(query,model) #preprocessing the input

    x=[]
    # calculate the cosine similarities of input query with all the vector abstracts
    for i in range(len(p)):
        x.append(cos_sim(query,p[i]))

    temp = list(x)

    # sort the list to find the top n similar results
    res = sorted(range(len(x)),key=lambda sub: x[sub])[-5:] #This code gets the position of the highest similarity values

    simi = [temp[i] for i in reversed(res)] #This line is to get the similarity values based on the position
    print(simi)

    L = []
    for i in reversed(res):
        L.append(i)

    return df1.iloc[L,[0,1,2,3,5]],simi
    

In [134]:
# Skipgram model results
query = "patient"
p = skipgram_100_vectors
model = skipgram_100
Results,sim = top_n(query,p,df1,model)
Results

C:\Users\Gyegn\AppData\Local\Temp\ipykernel_30972\923868345.py:2: RuntimeWarning: invalid value encountered in double_scalars
  return round(dot(a,b)/(norm(a)*norm(b)),3)


[0.805, 0.804, 0.792, 0.782, 0.782]


,Date added,Trial ID,Title,Brief title,Abstract
6142,9/19/2020,NCT04555096,"A Pilot, Randomized, Placebo-Controlled Trial of GC4419 (Avasopasem Manganese) in Patients With Critical Illness Due to SARS-CoV-2 Infection (COVID-19)",A Trial of GC4419 in Patients With Critical Illness Due to COVID-19,A Trial of GC4419 in Patients with Critical Illness due to COVID-19
5713,10/20/2020,NCT04591704,The Effect of Diabetes Mellitus on the Morbidity and Mortality Rates in Patients With COVID-19,The Effect of Diabetes Mellitus on the Prognosis of Patients With COVID-19,"All hospitalised patients with COVID-19 who have positive RT-PCR for SARS-COV-2 will be included in the study.The patients will be divided into two groups, as diabetics and non-diabetics. The COVID-19 patients' medical records will be evaluated and compared in terms of the duration of hospitalization, the presence of lung involvement in Computerised Tomography, the need for intensive care unit and mortality rates in patients with and without diabetes.\n\nDetailed Description\nAll hospitalised patients with COVID-19 who have positive RT-PCR for SARS-COV-2 will be included in the study.The patients will be divided into two groups, as diabetics and non-diabetics. The diagnosis of diabetes mellitus will be extracted from the medical records and medical history of the patients hospitalised with COVID-19.The COVID-19 patients medical records will be evaluated and compared in terms of the duration of hospitalization, the presence of lung involvement in Computerised Tomography, the need for intensive care unit and mortality rates in patients with and without diabetes."
10621,4/16/2020,NCT04346797,"CORIMUNO19-ECU: Trial Evaluating Efficacy and Safety of Eculizumab (Soliris) in Patients With COVID-19 Infection, Nested in the CORIMUNO-19 Cohort","CORIMUNO19-ECU: Trial Evaluating Efficacy and Safety of Eculizumab (Soliris) in Patients With COVID-19 Infection, Nested in the CORIMUNO-19 Cohort","The overall objective of the study is to determine the therapeutic effect and tolerance of Eculizumab in patients with moderate, severe pneumonia or critical pneumonia associated with Coronavirus disease 2019 (COVID-19). Eculizumab is a terminal complement inhibitor that has been investigated for more than 10 years in numerous complement-mediated diseases. The study has a cohort multiple Randomized Controlled Trials (cmRCT) design. Randomization will occur prior to offering Eculizumab administration to patients enrolled in the CORIMUNO-19 cohort. Eculizumab will be administered to consenting adult patients hospitalized with COVID-19 either diagnosed with moderate or severe pneumonia requiring no mechanical ventilation or critical pneumonia requiring mechanical ventilation. Patients who will chose not to receive Eculizumab will receive standard of care. Outcomes of Eculizumab-treated patients will be compared with outcomes of standard of care-treated patients as well as with outcomes of patients treated with other immune modulators."
11554,3/14/2020,CHICTR2000030744,Clinical Application of ECMO(or Ultra-Protective Lung Mechanical Ventilation) in the Treatment of Patients with ARDS due to novel Coronavirus Pneumonia (COVID-19),Clinical Application of ECMO(or Ultra-Protective Lung Mechanical Ventilation) in the Treatment of Patients with ARDS due to novel Coronavirus Pneumonia (COVID-19),"This study intends to divide patients with ARDS due to novel Coronavirus Pneumonia (COVID-19) into two groups: the ECMO group and the conventional treatment group,observe whether ECMO can improve the outcome of such patients."
4432,12/17/2020,NCT04668911,"Oral Health, Microbial Burden and COVID-19","Oral Health, Microbial Burden and COVID-19","An observational study of patients with COVID-19 confirmed cases (with various degrees of severity) and controls.Oral and nasal swabs will be taken from 150 patients (50 with mild form and 50 with severe form of COVID-19 with or without mechanical ventilation, 50 healthy cont

In [113]:
# Now lets try with fasttext vector abstracts
vector_fasttext_100 = pd.read_csv("k_fasttext_abstract.csv")
vector_fasttext_100 = vector_fasttext_100.T
fasttext_100_vectors = []
for i in range(df1.shape[0]):
    fasttext_100_vectors.append(vector_fasttext_100[i].values)

In [122]:
fasttext_100_vectors = []
for i in range(df1.shape[0]):
    fasttext_100_vectors.append(vector_fasttext_100[i].values)

In [133]:
query = "patient"
p = fasttext_100_vectors
model = fast_n_100
Results,sim = top_n(query,p,df1,model)
Results

C:\Users\Gyegn\AppData\Local\Temp\ipykernel_30972\923868345.py:2: RuntimeWarning: invalid value encountered in double_scalars
  return round(dot(a,b)/(norm(a)*norm(b)),3)


[0.918, 0.912, 0.909, 0.901, 0.9]


,Date added,Trial ID,Title,Brief title,Abstract
11087,4/4/2020,CHICTR2000031587,"A paired clinical study of novel coronavirus pneumonia (COVID-19) patients with ordinary, severe, critical, and deceased patients in Tongji Hospital","A paired clinical study for novel coronavirus pneumonia (COVID-19) patients with ordinary, severe, critical, and deceased patients in Tongji Hospital","A paired clinical study of COVID - 19 patients with ordinary, severe, critical, and deceased patients in Wuhan"
2220,4/21/2021,2020-005214-18,An Investigation of the Efficacy and Safety of Favipiravir in COVID-19 Patients without Pneumonia An open-label randomized controlled study,An Investigation of the Efficacy and Safety of Favipiravir in COVID-19 Patients without Pneumonia,COVID-19 Patients without Pneumonia
5713,10/20/2020,NCT04591704,The Effect of Diabetes Mellitus on the Morbidity and Mortality Rates in Patients With COVID-19,The Effect of Diabetes Mellitus on the Prognosis of Patients With COVID-19,"All hospitalised patients with COVID-19 who have positive RT-PCR for SARS-COV-2 will be included in the study.The patients will be divided into two groups, as diabetics and non-diabetics. The COVID-19 patients' medical records will be evaluated and compared in terms of the duration of hospitalization, the presence of lung involvement in Computerised Tomography, the need for intensive care unit and mortality rates in patients with and without diabetes.\n\nDetailed Description\nAll hospitalised patients with COVID-19 who have positive RT-PCR for SARS-COV-2 will be included in the study.The patients will be divided into two groups, as diabetics and non-diabetics. The diagnosis of diabetes mellitus will be extracted from the medical records and medical history of the patients hospitalised with COVID-19.The COVID-19 patients medical records will be evaluated and compared in terms of the duration of hospitalization, the presence of lung involvement in Computerised Tomography, the need for intensive care unit and mortality rates in patients with and without diabetes."
11523,3/15/2020,CHICTR2000030803,Clinical characteristics of severe or critically ill patients infected with 2019-nCoV,Collection and analysis of clinical data in severe and critically ill patients with novel coronavirus pneumonia (COVID-19),"we performed an observational study to comprehensively evaluate the clinical characteristics of severe or critically ill patients with COVID-19 based on clinical data, and aimed to provide the scientific evidence for optimal treatment and management of severe or critically ill patients with COVID-19."
11943,2/26/2020,CHICTR2000029757,Convalescent plasma for the treatment of severe and critical novel coronavirus pneumonia (COVID-19): a prospective randomized controlled trial,Convalescent plasma for the treatment of severe and critical novel coronavirus pneumonia (COVID-19): a prospective randomized controlled trial,"In order to actively prevent and control COVID-19, the convalescent plasma of patients with COVID-19 is collected, and the clinical treatment plan of using convalescent plasma to treat patients with COVID-19 is explored."


# Both the models perform well with good similarity results.